In [ ]:
import concurrent.futures
import os
from functools import partial

import matplotlib.pyplot as plt
import numpy as np

from materials import get_materials_Cvec
from safe_module import distance
from themes import color_themes
from themes import symmetry_classes
from utilities import read_table

In [ ]:
write = False

In [ ]:
colors = color_themes()

In [ ]:
# Analysis for -
# 1. - Brown2016 Table 2 materials (Brown)
# 2. - BrownAbramson2016 Table 3 materials (BrownAbramson)

study = "Brown"
data_size = 1000
compute = False

In [ ]:
if study=="Brown":
    materials = ['An0', 'An25', 'An37', 'An48', 'An60', 'An67', 'An78', 'An96']
    n_materials = len(materials)
    anorthite_percentages = [0, 25, 37, 48, 60, 67, 78, 96]
    legend_title = "Brown2016"
    data_dir = "Brown2016"

elif study=="BrownAbramson":
    materials = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
    n_materials = len(materials)
    filename = "../data/BrownAbramson2016_Table2.txt"
    composition = read_table(filename)
    total_aluminium = np.zeros(n_materials)
    for i in range(n_materials):
        total_aluminium[i] = float(composition[1][i+1]) + float(composition[2][i+1])
    legend_title = "BA2016"
    data_dir = "BrownAbramson2016"

In [ ]:
workers = os.cpu_count()
n_symmetry_classes = len(symmetry_classes)
vecs = get_materials_Cvec(material=study)
c_vecs = [vecs[i] for i in np.arange(0,n_materials*2,2)]
stdv_vecs = [vecs[i] for i in np.arange(1,n_materials*2,2)]

In [ ]:
means = np.empty((n_symmetry_classes, n_materials))
stdvs = np.empty((n_symmetry_classes, n_materials))

In [ ]:
# Section to find beta values for the realizations

if compute:
    for i, material in enumerate(materials):
        
        print(f'running analysis for material {material}')
        
        if write:
            with open('status.txt', 'a') as f:
                f.write(f'running analysis for material {material}\n')
            
            with open(f'output_{material}.txt', 'w') as f:
                f.write('C11 \tC12 \tC13 \tC14 \tC15 \tC16 \tC22 \tC23 \tC24 \tC25 \tC26 \tC33 \tC34 \tC35 \tC36 \tC44 '
                 '\tC45 \tC46 \tC55 \tC56 \tC66\n')
    
        cvecs = []
    
        for _ in range(data_size):
            c_vec_random = np.around( np.random.normal( c_vecs[i], stdv_vecs[i]), 1 )
            if write:
                with open(f'output_{material}.txt', 'a') as f:
                    for value in c_vec_random:
                        f.write(f'{value:.1f} \t')
                    f.write('\n')
            cvecs.append(c_vec_random)
            
        betas = np.empty((n_symmetry_classes, data_size))
    
        distance_new = partial(distance, use_parallel_processing=False)
    
        for j, sigma in enumerate(symmetry_classes):   
            print(f'running minimizer for symmetry class: {sigma}')
            if write:
                with open('status.txt', 'a') as f:
                    f.write(f'running minimizer for symmetry class: {sigma}\n')
            with concurrent.futures.ProcessPoolExecutor(max_workers=workers) as executor:
                results = list(executor.map(distance_new, cvecs, [sigma]*data_size))
            for k in range(data_size):
                betas[j,k] = results[k][0]
        
            means[j,i] = np.mean(betas[j,:])
            stdvs[j,i] = np.std(betas[j,:])
            
        if write: np.save(f'betas_brown_{material}_1000', betas)
        
else:
    
    for i, material in enumerate(materials):
        betas = np.load(f"Data/{data_dir}/betas_brown_{material}_{data_size}.npy")
        for j in range(n_symmetry_classes):   
            means[j,i] = np.mean(betas[j,:])
            stdvs[j,i] = np.std(betas[j,:])

In [ ]:
if study=="Brown":
    clrs = ['red', 'orange', 'teal', 'cyan', 'blue', 'purple', 'grey', 'black']
elif study=="BrownAbramson":
    clrs = ['red', 'orange', 'teal', 'cyan', 'blue', 'purple', 'brown', 'grey', 'black']
    
plt.figure(figsize=(7,10))
x = np.arange(n_symmetry_classes)
for i, material in enumerate(materials):
    plt.plot(x, means[:,i], '-o', label=material, color=clrs[i])
    plt.errorbar(x, means[:,i], yerr=2*stdvs[:,i], color=clrs[i], capsize=5, capthick=1)
plt.xticks(x, symmetry_classes, fontsize=15)
plt.yticks(fontsize=15)
plt.legend(loc=1, fontsize=13, title=f"{legend_title}", title_fontsize=15)
plt.grid()
plt.xlabel(r'${\Sigma}$', fontsize=20)
plt.ylabel(r'$\beta_{\Sigma}$ (°)', fontsize=20)
if write: plt.savefig('1.png', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(7,10))

if study=="Brown":
    x = anorthite_percentages
    for i, symmetry_class in enumerate(symmetry_classes):
        plt.plot(x, means[i], '-o', color=colors[symmetry_class], label=symmetry_class)
        plt.errorbar(x, means[i], yerr=2*stdvs[i], color=colors[symmetry_class], capsize=5, capthick=1)
    plt.xlabel('Anorthite percentage', fontsize=20)
    plt.xticks(np.arange(0,101,10), fontsize=15)
    plt.legend(loc=1, fontsize=13, title="${\Sigma}$"+f" ({legend_title})", title_fontsize=15)
    
elif study=="BrownAbramson":    
    x = total_aluminium
    index = np.argsort(x)
    for i, symmetry_class in enumerate(symmetry_classes):
        plt.plot(x[index], means[i][index], '-o', color=colors[symmetry_class], label=symmetry_class)
        plt.errorbar(x[index], means[i][index], yerr=2*stdvs[i][index], color=colors[symmetry_class], capsize=5, capthick=1)
    plt.xlabel('Total aluminium (atoms/F.U.)', fontsize=20)
    plt.xticks(fontsize=15)
    plt.legend(loc=1, fontsize=10, title='${\Sigma}$ (BA2016)', title_fontsize=10)

plt.yticks(fontsize=15)
plt.grid()
plt.ylabel(r'$\beta_{\Sigma}$ (°)', fontsize=22)
if write: plt.savefig('2.png', bbox_inches='tight')